In [133]:
#required packages and libraries
import numpy as np 
import matplotlib.pyplot as plt #for plotting
import seaborn as sns 
%matplotlib inline
from statsmodels.stats.power import TTestIndPower #Power calculatons
import ipywidgets  #for creating interactive sliders
import warnings
warnings.filterwarnings('ignore')

### Basic sample size determination using power analysis

This notebook provides an interactive plot which allows for the exploration of sample sizes required for desired significance level, power and effect size in a study. It is a very simple method which basically looks up the sample size needed to achieve a certain power and significance level conditional on the effect size. The goal is to build upon this basic approach to make suitable in the context of metabolic studies.

### Preliminaries

**Power analysis** is a technique so enough samples are taken in a study to identify a significant effect but not too large to be wasteful.

#### Some key terminology:
- The **significance** $(\alpha)$ of a test is the probability of saying a non-effect exists.

- The **power** $(1-\beta)$ of a test is the probability of saying a effect exists if in fact it exists

- The **effect size** is the degree to which a phenomenon exists in a population. 

Take for example testing an athlete for doping where the number of samples is the number of biomarkers being tested for irregularities. The effect size here is the degree to which one of these biomarkers differs from that of an individual who is completely clean, never doped. The goal is to take enough samples so that we can say with an acceptable degree of confidence the doping status of such an athlete. Ideally we want to take as little samples as possible due to cost constraints etc. 

If we set the signifacnce level of a test really low, say 0.001, so that we almost never make the mistake of saying a clean athlete has doped then it is quite likely that we end up in the scenario that we also rearely pick up a doped athlete as doped. ie. low power of the test. (include plots for these)

Moreover, if we set the significance level to some reasonable value and desire extremely high power, say 0.95, then in order to achieve such a power we need to take an unreasonably large sample size. 

Conventionally the power of a test is taken to be 0.8 (0.2 of the time we do not identify an effect) and significance at 0.05. This does not seem to differ in metabolic studies (Blaise et al. etc.). This implies we assume falsely saying a clean athlete has doped is 4 times worse than saying a doped athlete is clean in the above example. 

In [ ]:
def n_es_plot(alpha = 0.05, power = 0.8, es_in = 0.3):
    
    """
    Funtion for creating a plot for given signifiacnce and power
    """
    
    effect_sizes = np.arange(0.005, 2, 0.005) #Range of effect sizes
    
    #Empty lists for storing results
    es_list = []
    n_list =[]
    for es in effect_sizes:
        power_analysis = TTestIndPower() #uses pooled variances
        n_needed = power_analysis.solve_power(effect_size = es, 
                                               alpha = alpha,
                                               power = power,
                                               alternative = "two-sided") #Calculate the power of a t-test for two independent sample
        es_list.append(es)
        n_list.append(n_needed)
        
    n_in = n_list[es_list.index(es_in)]
    
    sns.set_style('darkgrid')
    plt.plot(n_list, es_list, '-', color = 'mediumorchid')
    plt.scatter(n_in, es_in, c="midnightblue")
    plt.plot([n_in, n_in], [0,es_in], c = "slateblue", linestyle = "--")
    plt.plot([0, n_in], [es_in, es_in], c = "slateblue", linestyle = "--")
    plt.xlim([0, 500]) #setting limits on x
    plt.xlabel("Sample Size")
    plt.ylabel("Effect Size")
    plt.title(f'Sample size vs. effect size for \n significance {alpha} and power {power}')
    annotation = plt.annotate(text = f'{round(n_in)} samples', 
                xy = (n_in,0),
                xytext = (15, 15),
                              color = 'w',
                textcoords = 'offset points',
                bbox = {'boxstyle': 'round', 'fc':'mediumorchid'},
                arrowprops = {'arrowstyle':'->', 'color': "mediumorchid"}
                             )
    annotation.set_visible(True)
    plt.grid(True, linestyle = '--')
    

In [ ]:
ipywidgets.interact(n_es_plot, alpha = (0, 0.1, 0.01), power = (0,1, 0.01), es_in = (0, 2, 0.05))

The above default plot implies we need 175 samples per group (total of 350) to have an 80% chance of detecting a difference of 0.3 units.

Blaise et al. (2016) proposed a method to apply power analysis to metabolic data given a small cohort of pilot data via bootstrap simulation to get an accurate estimate of the effect size expected from these metabolites in a bigger study to in turn aid in estimating the sample size needed to achieve a certain level of power and significance in the study overall. 

The above simple method just lets the researcher decide what an expected efect size should be and simply read off the sample size needed to achieve a significance of 0.05 and power of 0.8 with that effect size. 

Note the goal is to collect a large enough sample to have sufficient power to detect a meaningful effect—but not too large to be wasteful.

It uses a two-sampled t-test with pooled variance

